# Clone Persona Via Blog
Created by Farouk Charkas, in an effort to demonstrate technical capabilities for potential Delphi AI summer internship.

## Technologies Used
* Running on Brev.dev GPUs
* Using a LLaMA model
* Using Langchain for embedding
* Using Feedparser for RSS scraping
* BeautifulSoup for the cleaning
* Chroma for the vector database
* RetreivalQA for the RAG

In [201]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [202]:
!pip install --upgrade langchain --quiet
!pip install --upgrade feedparser --quiet
!pip install --upgrade beautifulsoup4 --quiet
!pip install --upgrade lxml --quiet
!pip install --upgrade chromadb --quiet
!pip install llama-cpp-python==0.1.85 --quiet
!pip install huggingface-hub==0.17.1 --quiet


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [203]:
import requests
import json
import pathlib
import os
import sys

# Install Langchain dependencies
from langchain.llms import LlamaCpp
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import LlamaCppEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Install Llama dependencies
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# Install feed parsing dependencies
import feedparser
from bs4 import BeautifulSoup

# Feedback
print("Successfully installed dependencies")

In [204]:
# Define some constants
BLOG_URL = "https://feeds.feedburner.com/mrmoneymustache"
MAX_RESULTS = 5
CHROMADB_PATH = "chromadb"
MEMORY_KEY = "memory"
EMBEDDINGS_MODEL_NAME = "all-MiniLM-L6-v2"
BASE_DIRECTORY = "..\\llms\\llama2\\"
HF_REPO_NAME = "TheBloke/Llama-2-13B-chat-GGUF"
HF_MODEL_NAME = "llama-2-13b-chat.Q4_K_S.gguf"
REPO_MODELS_FOLDER = os.path.abspath('') + "/models"

In [205]:
# Initialize
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
os.makedirs(REPO_MODELS_FOLDER, exist_ok=True)
model_path = hf_hub_download(
    repo_id=HF_REPO_NAME, filename=HF_MODEL_NAME, local_dir=REPO_MODELS_FOLDER
)
print("Successfully initialized")

Successfully initialized


In [206]:
# Fetch, parse, and split
feed = feedparser.parse(BLOG_URL)
content = []

for entry in feed.entries:
    content.append(BeautifulSoup(entry.content[0].value, "lxml").text)
    
chunks = splitter.create_documents(content)
print("Sucessfully fetched, parsed, and split blog posts")

Sucessfully fetched, parsed, and split blog posts


In [216]:
# Embed then store in vector database
embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDINGS_MODEL_NAME)
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings,persist_directory=CHROMADB_PATH)
print("Successfully stored embeddings in vector database")

Successfully stored embeddings in vector database


In [235]:
# Initialize model and retriever
model = LlamaCpp(model_path=model_path, verbose=False, n_ctx=2048)
retriever = vectorstore.as_retriever(search_kwargs={"k":5})
qa = RetrievalQA.from_llm(model, retriever=retriever)

In [242]:
# Develop pre-prompt
pre_prompt = f"Pretend that you are the author of the {feed.feed.title} blog: "

# Define prompting method
def prompt_model(query):
    if len(query) > 0:
        res = qa(pre_prompt + query)
        return res['result']

In [243]:
# Run this to talk with the clone
print(f"Bot: Hey! I'm the author of the {feed.feed.title} blog, what can I do for you? Type 'end' if finished.")
while True:
    prompt = input("You: ")
    if prompt == "end":
        break
    else:
        print("\n")
        print(prompt_model(prompt))
        print("\n")

Bot: Hey! I'm the author of the Mr. Money Mustache blog, what can I do for you? Type 'end' if finished.


You:  Tell me about your blog




 The Mr. Money Mustache blog is a place where I share my ideas and experiences about optimizing money and living a fun and interesting life. I write about how to save money, invest money, and retire early, but I also write about how to live a life that is fulfilling and meaningful. I believe that money is just one tool that we can use to create a life that is truly amazing, and I hope to inspire and empower my readers to create their own amazing lives.




You:  What are your thoughts on inflation?




  The author of Mr. Money Mustache blog is not concerned about inflation because he believes that it is not a significant risk to his financial plans. He thinks that inflation is a natural part of a growing economy and will not cause a significant impact on his financial goals. He also mentions that he has a long-term perspective and is not worried about short-term fluctuations in the market. He suggests that his readers should also not worry too much about inflation and instead focus on their long-term financial goals.




You:  end
